Import Libraries

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')

Import Simulated Loss Data

In [2]:
loss_data = pd.read_csv('dev/_baseline_exercises/SimulatedLossData.csv') #Import simulated loss data

In [3]:
# Set float format for all floats in the DataFrame display
pd.options.display.float_format = '{:.2f}'.format

In [4]:
# Import and categorize specific peril codes
peril_codes_mapping = {
    1: ['WS', 'Windstorm'], 
    2: ['EQ', 'Earthquake'], 
    3: ['CS', 'Sever Convective Storm'], 
    9: ['WF', 'Willoss_dataire']
}

In [5]:
loss_data['PerilCode'] = loss_data['PerilID'].map(lambda x: peril_codes_mapping[x][0])
loss_data['PerilName'] = loss_data['PerilID'].map(lambda x: peril_codes_mapping[x][1])

Gross AAL

In [6]:
total_aal = loss_data['Loss'].sum()
Year_count = loss_data['Year'].nunique()
gross_aal = total_aal / Year_count
print(f"Gross Average Annual Loss: ${gross_aal:,.2f}")

Gross Average Annual Loss: $140,653,378.97


Net AAL

In [7]:
def allocate_to_layers(loss):
    # Define each layer with (attachment_point, limit)
    retention_attach = 0
    retention_limit = 10e6
    
    working_attach = 10e6
    working_limit = 40e6  # Covers from $10M to $50M
    
    layer1_attach = 50e6
    layer1_limit = 50e6  # Covers from $50M to $100M
    
    seawall_attach = 100e6
    seawall_limit = 100e6  # Covers from $100M to $200M
    
    catbond_attach = 200e6
    catbond_limit = 100e6  # Covers from $200M to $300M

    super_cat_attach = 300e6
    super_cat_limit = 500e6
    
    # Initialize result
    result = {
        'retention_paid': 0,
        'working_paid': 0,
        'layer1_paid': 0,
        'seawall_paid': 0,
        'catbond_paid': 0,
        'supercatbond_paid':0,
        'excess_loss': 0
    }

    result['retention_paid'] = min(loss, retention_limit)

    if loss > working_attach:
        excess_over_attach = loss - working_attach
        result['working_paid'] = min(excess_over_attach, working_limit)
    
    if loss > layer1_attach:
        excess_over_attach = loss - layer1_attach
        result['layer1_paid'] = min(excess_over_attach, layer1_limit)

    if loss > seawall_attach:
        excess_over_attach = loss - seawall_attach
        result['seawall_paid'] = min(excess_over_attach, seawall_limit)

    if loss > catbond_attach:
        excess_over_attach = loss - catbond_attach
        result['catbond_paid'] = min(excess_over_attach, catbond_limit)

    if loss > super_cat_attach:
        excess_over_attach = loss - super_cat_attach
        result['catbond_paid'] = min(excess_over_attach, super_cat_limit)

    max_coverage = super_cat_attach + super_cat_limit 
    if loss > max_coverage:
        result['excess_loss'] = loss - max_coverage

    result['net_loss'] = result['retention_paid'] + result['excess_loss']

    return result


In [8]:
layers = allocate_to_layers(int(gross_aal))

In [9]:
kpi_dataset = pd.DataFrame(
    {"metric": ["gross_aal", "retention_paid", "working_paid", "layer1_paid", "seawall", "catbond_paid", "supercatbond_paid", "excess_loss"],
     "value": [gross_aal, layers['retention_paid'], layers['working_paid'], layers['layer1_paid'], layers['seawall_paid'], layers['catbond_paid'], layers['supercatbond_paid'], layers['excess_loss']]}
)

In [10]:
kpi_dataset['value_type'] = 'kpi'

In [11]:
kpi_dataset

,metric,value,value_type
0,gross_aal,140653378.97,kpi
1,retention_paid,10000000.00,kpi
2,working_paid,40000000.00,kpi
3,layer1_paid,50000000.00,kpi
4,seawall,40653378.00,kpi
5,catbond_paid,0.00,kpi
6,supercatbond_paid,0.00,kpi
7,excess_loss,0.00,kpi


In [12]:
kpi_dataset = kpi_dataset[['value_type', 'metric', 'value']]

Running Loss Curve Scenarios

In [13]:
def generate_loss_curve(annual_total_loss, annual_max_events, return_periods):
    aggregate_loss_curve = []
    max_event_loss_curve = []

    for rp in return_periods:
        percentile = 100 * (1 - 1/rp)
        agg_loss = np.percentile(annual_total_loss['annual_loss'], percentile)
        occ_loss = np.percentile(annual_max_events['max_event_loss'], percentile)

        aggregate_loss_curve.append(agg_loss)
        max_event_loss_curve.append(occ_loss)
    
    return aggregate_loss_curve, max_event_loss_curve

In [14]:
return_periods = [2, 5, 10, 25, 50, 100, 130, 200, 250, 500, 1000, 5000, 10000]


In [15]:
annual_total_loss = loss_data.groupby('Year').agg(annual_loss=('Loss', 'sum')).reset_index()
annual_max_events = loss_data.groupby('Year').agg(max_event_loss=('Loss', 'max')).reset_index()

In [16]:
annual_total_loss = loss_data.groupby('Year').agg(annual_loss=('Loss', 'sum')).reset_index()
annual_max_events = loss_data.groupby('Year').agg(max_event_loss=('Loss', 'max')).reset_index()

loss_curves_df = pd.DataFrame({
    'Return Period': return_periods
})
loss_curves_df['Percentile'] = loss_curves_df['Return Period'].apply(lambda x: 100 * (1 - 1/x))

portfolio_agg_loss_curve, portfolio_occ_loss_curve = generate_loss_curve(annual_total_loss, annual_max_events, return_periods)
loss_curves_df[f'agg_loss_portfolio'] = portfolio_agg_loss_curve
loss_curves_df[f'max_event_loss_portfolio'] = portfolio_occ_loss_curve

In [17]:
loss_curves_df

,Return Period,Percentile,agg_loss_portfolio,max_event_loss_portfolio
0,2,50.00,66399232.34,26565450.00
1,5,80.00,176702883.67,98416540.00
2,10,90.00,305936025.87,192967600.00
3,25,96.00,549791416.73,384580600.00
4,50,98.00,793930812.58,592938980.00
5,100,99.00,1155379507.78,941963300.00
6,130,99.23,1324428720.13,1084841307.69
7,200,99.50,1722248858.92,1513910850.00
8,250,99.60,1995053234.92,1662068120.00
9,500,99.80,3017348594.58,2618025780.00


In [18]:
net_loss = []
for i in loss_curves_df['agg_loss_portfolio']:
    nl_calc = allocate_to_layers(int(i))['net_loss']
    net_loss.append(nl_calc)

loss_curves_df['agg_net_loss'] = net_loss
    

In [19]:
tvar_kpi = allocate_to_layers(int(loss_curves_df['agg_loss_portfolio'][8]))['net_loss']

In [20]:
kpi_dataset.loc[len(kpi_dataset)] = ["kpi", "tvar_net_loss", tvar_kpi]

In [21]:
kpi_dataset.loc[len(kpi_dataset)] = ["kpi", "tvar_gross_loss", int(loss_curves_df['agg_loss_portfolio'][8])]

In [22]:
kpi_dataset

,value_type,metric,value
0,kpi,gross_aal,140653378.97
1,kpi,retention_paid,10000000.00
2,kpi,working_paid,40000000.00
3,kpi,layer1_paid,50000000.00
4,kpi,seawall,40653378.00
5,kpi,catbond_paid,0.00
6,kpi,supercatbond_paid,0.00
7,kpi,excess_loss,0.00
8,kpi,tvar_net_loss,1205053234.00
9,kpi,tvar_gross_loss,1995053234.00


In [24]:
loss_curves_df

,Return Period,Percentile,agg_loss_portfolio,max_event_loss_portfolio,agg_net_loss
0,2,50.00,66399232.34,26565450.00,10000000.00
1,5,80.00,176702883.67,98416540.00,10000000.00
2,10,90.00,305936025.87,192967600.00,10000000.00
3,25,96.00,549791416.73,384580600.00,10000000.00
4,50,98.00,793930812.58,592938980.00,10000000.00
5,100,99.00,1155379507.78,941963300.00,365379507.00
6,130,99.23,1324428720.13,1084841307.69,534428720.00
7,200,99.50,1722248858.92,1513910850.00,932248858.00
8,250,99.60,1995053234.92,1662068120.00,1205053234.00
9,500,99.80,3017348594.58,2618025780.00,2227348594.00


Ouput datasets

In [23]:
kpi_dataset.to_excel("kpi_dataset.xlsx")

In [25]:
loss_curves_df.to_excel("loss_curves.xlsx")